<a href="https://colab.research.google.com/github/cssmil/AppJest/blob/master/Inteligencia_Artificial_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Clasificación de imágenes con el conjunto de datos MNIST

En esta sección haremos el «Hola Mundo» del Deep Learning: entrenar un modelo de para clasificar correctamente los dígitos escritos a mano.

## 1.1 Objetivos

* Comprender cómo el Deep Learningpuede resolver problemas que los métodos de programación tradicionales no pueden.
* Concoer el [MNIST handwritten digits dataset](http://yann.lecun.com/exdb/mnist/)
* Utilizar [torchvision](https://pytorch.org/vision/stable/index.html) para cargar el conjunto de datos MNIST y prepararlo para el entrenamiento.
* Crear una red neuronal simple para realizar la clasificación de imágenes.
* Entrene la red neuronal utilizando el conjunto de datos MNIST preparado.
* Observar y Analizar el rendimiento de la red neuronal entrenada.

Comencemos cargando las bibliotecas utilizadas en este cuaderno:

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

# Visualization tools
import torchvision
import torchvision.transforms.v2 as transforms
import torchvision.transforms.functional as F
import matplotlib.pyplot as plt

En PyTorch, podemos utilizar nuestra GPU en nuestras operaciones estableciendo el [device](https://pytorch.org/docs/stable/tensor_attributes.html#torch.device) a `cuda`. La función `torch.cuda.is_available()` confirmará que PyTorch puede reconocer la GPU.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()

True

### 1.1.1 El Problema: Clasificación de Imágenes

En la programación tradicional, el programador es capaz de articular reglas y condiciones en su código que su programa puede utilizar para actuar de la forma correcta. Este enfoque sigue funcionando excepcionalmente bien para una gran variedad de problemas.

La clasificación de imágenes, que pide a un programa que clasifique correctamente una imagen que nunca ha visto antes en su clase correcta, es casi imposible de resolver con las técnicas de programación tradicionales. ¿Cómo podría un programador definir las reglas y condiciones para clasificar correctamente una enorme variedad de imágenes, sobre todo teniendo en cuenta imágenes que nunca ha visto?

### 1.1.2 La Solución: Deep Learning

El Deep Learning destaca en el reconocimiento de patrones por ensayo y error. Al entrenar una red neuronal profunda con datos suficientes y proporcionar a la red información sobre su rendimiento a través del entrenamiento, la red puede identificar, a través de una enorme cantidad de iteraciones, su propio conjunto de condiciones por las que puede actuar de la manera correcta.

## 1.2 El conjunto de datos MNIST

En la historia del Deep Learning, la clasificación precisa de imágenes del [conjunto de datos MNIST](http://yann.lecun.com/exdb/mnist/), una colección de 70 000 imágenes en escala de grises de dígitos manuscritos del 0 al 9, supuso un gran avance. Aunque hoy en día el problema se considera trivial, realizar la clasificación de imágenes con MNIST se ha convertido en una especie de «Hola mundo» para el Deep Learning.

Aquí se muestran 100 de las imágenes incluidas en el conjunto de datos MNIST:

<img src="https://upload.wikimedia.org/wikipedia/commons/f/f7/MnistExamplesModified.png" style="width: 600px;">

### 1.2.1 Datos y etiquetas de entrenamiento y validación

Cuando trabajamos con imágenes para el aprendizaje profundo, necesitamos tanto las imágenes en sí, normalmente denominadas «X», como las [etiquetas correctas](https://developers.google.com/machine-learning/glossary#label) para estas imágenes, normalmente denominadas «Y». Además, necesitamos los valores `X` y `Y` para *entrenar* el modelo y, a continuación, un conjunto separado de valores `X` y `Y` para *validar* el rendimiento del modelo una vez entrenado.

Podemos imaginar estos pares `X` y `Y` como un conjunto de fichas. Un alumno puede entrenarse con un conjunto de fichas y, para validar que ha aprendido los conceptos correctos, el profesor puede preguntarle con otro conjunto de fichas.

Por lo tanto, necesitamos 4 segmentos de datos para el conjunto de datos MNIST:

1. `x_train`: Imágenes utilizadas para el entrenamiento de la red neuronal
2. `y_train`: Etiquetas correctas para las imágenes `x_train`, utilizadas para evaluar las predicciones del modelo durante el entrenamiento
3. `x_valid`: Imágenes reservadas para validar el rendimiento del modelo una vez entrenado.
4. `y_valid`: Etiquetas correctas para las imágenes `x_valid`, utilizadas para evaluar las predicciones del modelo una vez entrenado.

El proceso de preparación de los datos para el análisis se denomina [Ingeniería de datos](https://medium.com/@rchang/a-beginners-guide-to-data-engineering-part-i-4227c5c457d7). Para saber más sobre las diferencias entre datos de entrenamiento y datos de validación (así como datos de prueba), consulte [este artículo](https://machinelearningmastery.com/difference-test-validation-datasets/) de Jason Brownlee.

### 1.2.2 Cargar los datos en memoria (con Keras)

Hay muchos [frameworks de Deep Learning](https://developer.nvidia.com/deep-learning-frameworks), cada uno con sus propios méritos. En este taller trabajaremos con [PyTorch 2](https://pytorch.org/get-started/pytorch-2.0/), y específicamente con la [Sequential API](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html). La API Secuencial tiene muchas funciones útiles diseñadas para construir redes neuronales. También es una opción legítima para el aprendizaje profundo en un entorno profesional debido a su [legibilidad](https://blog.pragmaticengineer.com/readable-code/) y la eficiencia, aunque no es el único en este sentido, y vale la pena investigar una variedad de marcos al iniciar un proyecto de aprendizaje profundo.

También utilizaremos la biblioteca [TorchVision](https://pytorch.org/vision/stable/index.html). Una de las muchas características útiles que ofrece son módulos que contienen métodos de ayuda para [muchos conjuntos de datos comunes](https://pytorch.org/vision/main/datasets.html), incluyendo MNIST.

Comenzaremos cargando los conjuntos de datos `train` y `valid` para [MNIST](https://pytorch.org/vision/main/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST).

In [ ]:
train_set = torchvision.datasets.MNIST("./data/", train=True, download=True)
valid_set = torchvision.datasets.MNIST("./data/", train=False, download=True)

Anteriormente dijimos que el conjunto de datos MNIST contenía 70.000 imágenes en escala de grises de dígitos manuscritos. Ejecutando las siguientes celdas, podemos ver que TorchVision ha particionado 60.000 de estas [Imágenes PIL](https://pillow.readthedocs.io/en/stable/reference/Image.html) para el entrenamiento, y 10.000 para la validación (después del entrenamiento).

In [ ]:
train_set

In [ ]:
valid_set

*Nota: El `Split` para `valid_set` se indica como `Test`, pero utilizaremos los datos para la validación en nuestros ejercicios prácticos. Para obtener más información sobre la diferencia entre los conjuntos de datos `Train`, `Valid` y `Test`, consulta [este artículo](https://kili-technology.com/training-data/training-validation-and-test-sets-how-to-split-machine-learning-data) de Kili.

### 1.2.3 Exploración de los datos MNIST


Tomemos el primer par `x, y` de `train_set` y revisemos las estructuras de datos:

In [ ]:
x_0, y_0 = train_set[0]

In [ ]:
x_0

In [ ]:
type(x_0)

¿Es un 5 o un 3 mal escrito? Podemos ver la etiqueta correspondiente para estar seguros.

In [ ]:
y_0

In [ ]:
type(y_0)

## 1.3 Tensores

Si un vector es una matriz unidimensional y una matriz es una matriz bidimensional, un tensor es una matriz n-dimensional que representa cualquier número de dimensiones. La mayoría de las redes neuronales modernas son potentes herramientas de procesamiento de tensores.

Un ejemplo de tensor tridimensional podrían ser los píxeles de una pantalla de ordenador. Las distintas dimensiones serían la anchura, la altura y el canal de color. Los videojuegos utilizan las matemáticas matriciales para calcular los valores de los píxeles de forma similar a como las redes neuronales calculan los tensores. Por eso, las GPU son eficaces máquinas de procesamiento de tensores.

Convirtamos nuestras imágenes en tensores para poder procesarlas después con una red neuronal. TorchVision tiene una función útil para convertir [Imágenes PIL](https://pillow.readthedocs.io/en/stable/reference/Image.html) en tensores con la clase [ToTensor](https://pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html):

In [ ]:
trans = transforms.Compose([transforms.ToTensor()])
x_0_tensor = trans(x_0)

Los tensores [PyTorch tensors](https://pytorch.org/docs/stable/tensors.html#torch.Tensor) tienen una serie de propiedades y métodos útiles. Podemos verificar el tipo de datos:

In [ ]:
x_0_tensor.dtype

Podemos verificar los valores mínimo y máximo. Las Imágenes PIL tienen un rango potencial entero de [0, 255], pero la clase [ToTensor](https://pytorch.org/vision/main/generated/torchvision.transforms.ToTensor.html) lo convierte a un rango flotante de [0.0, 1.0].

In [ ]:
x_0_tensor.min()

In [ ]:
x_0_tensor.max()

También podemos ver el tamaño de cada dimensión. PyTorch utiliza una convención `C x H x W`, lo que significa que la primera dimensión es el canal de color, la segunda es la altura, y la tercera es la anchura.

Como estas imágenes son en blanco y negro, sólo hay un canal de color. Las imágenes son cuadradas y tienen 28 píxeles de alto y ancho:

In [ ]:
x_0_tensor.size()

También podemos mirar los valores directamente:

In [ ]:
x_0_tensor

Por defecto, un tensor se procesa con una [CPU](https://www.arm.com/glossary/cpu).

In [ ]:
x_0_tensor.device

Para trasladarlo a una GPU, podemos utilizar el método `.cuda`.

In [ ]:
x_0_gpu = x_0_tensor.cuda()
x_0_gpu.device

El método `.cuda` fallará si PyTorch no reconoce una GPU. Para hacer nuestro código flexible, podemos enviar nuestro tensor `to` al `device` que identificamos al principio de este cuaderno. De esta manera, nuestro código se ejecutará mucho más rápido si una GPU está disponible, pero el código no fallará si no hay una GPU disponible.

In [ ]:
x_0_tensor.to(device).device

A veces, puede ser difícil interpretar tantos números. Afortunadamente, TorchVision puede convertir tensores `C x H x W` de nuevo en una imagen PIL con la función [to_pil_image](https://pytorch.org/vision/main/generated/torchvision.transforms.functional.to_pil_image.html).

In [ ]:
image = F.to_pil_image(x_0_tensor)
plt.imshow(image, cmap='gray')

## 1.4 Preparación de los datos para el entrenamiento


Anteriormente, creamos una variable `trans` para convertir una imagen en un tensor. [Transforms](https://pytorch.org/vision/stable/transforms.html) son un grupo de funciones de torchvision que pueden utilizarse para transformar un conjunto de datos.

### 1.4.1 Transformaciones


La función [Compose](https://pytorch.org/vision/stable/generated/torchvision.transforms.v2.Compose.html#torchvision.transforms.v2.Compose) combina una lista de transformaciones. Aprenderemos más sobre transformaciones en un cuaderno posterior, pero hemos copiado la definición `trans` a continuación como introducción.

In [ ]:
trans = transforms.Compose([transforms.ToTensor()])

Antes, sólo aplicábamos `trans` a un valor. Hay varias formas de aplicar nuestra lista de transformaciones a un conjunto de datos. Una de ellas es establecerla en la variable `transform` del conjunto de datos.

In [ ]:
train_set.transform = trans
valid_set.transform = trans

### 1.4.2 Cargadores de datos


Si nuestro conjunto de datos es una baraja de tarjetas flash, un [DataLoader](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html#preparing-your-data-for-training-with-dataloaders) define cómo extraemos tarjetas de la baraja para entrenar un modelo de IA. Podríamos mostrar a nuestros modelos todo el conjunto de datos a la vez. Esto no sólo requiere muchos recursos informáticos, sino que [las investigaciones demuestran](https://arxiv.org/pdf/1804.07612) que utilizar un lote de datos más pequeño es más eficaz para el entrenamiento de modelos.

Por ejemplo, si nuestro `batch_size` es 32, entrenaremos nuestro modelo barajando la baraja y sacando 32 cartas. No necesitamos barajar para la validación ya que el modelo no está aprendiendo, pero aún así utilizaremos un `batch_size` para evitar errores de memoria.

El tamaño del lote es algo que decide el desarrollador del modelo, y el mejor valor dependerá del problema que se esté resolviendo. La investigación muestra que 32 o 64 es suficiente para muchos problemas de aprendizaje automático y es el valor por defecto en algunos marcos de aprendizaje automático, por lo que utilizaremos 32 aquí.

In [ ]:
batch_size = 32

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size)

## 1.5 Creación del modelo

¡Es hora de construir el modelo! Las redes neuronales se componen de capas en las que cada capa realiza una operación matemática sobre los datos que recibe antes de pasarlos a la capa siguiente. Para empezar, crearemos un modelo de nivel «Hola Mundo» formado por 4 componentes:

1. Un [Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html) utilizado para convertir datos n-dimensionales en un vector.
2. Una capa de entrada, la primera capa de neuronas.
3. Una capa oculta, otra capa de neuronas «escondida» entre la entrada y la salida
4. Una capa de salida, el último conjunto de neuronas que devuelve la predicción final del modelo.

Puedes encontrar más información sobre estas capas en [esta entrada del blog](https://medium.com/@sarita_68521/basic-understanding-of-neural-network-structure-eecc8f149a23) de Sarita.

Vamos a crear una variable `layers` para contener nuestra lista de capas.

In [ ]:
layers = []
layers

### 1.5.1 Aplanar la imagen

Cuando miramos la forma de nuestros datos anteriores, vimos que las imágenes tenían 3 dimensiones: `C x H x W`. Aplanar una imagen significa combinar todas estas imágenes en 1 dimensión. Digamos que tenemos un tensor como el de abajo. Intenta ejecutar la celda de código para ver cómo se ve antes y después de ser aplanada.

In [ ]:
test_matrix = torch.tensor(
    [[1, 2, 3],
     [4, 5, 6],
     [7, 8, 9]]
)
test_matrix

In [ ]:
nn.Flatten()(test_matrix)

¿No ha pasado nada? Eso es porque las redes neuronales esperan recibir un lote de datos. Actualmente, la capa Aplanar ve tres vectores en lugar de una matriz 2D. Para solucionarlo, podemos «agrupar» nuestros datos añadiendo un par de corchetes extra. Desde `matriz_prueba` es ahora un tensor, podemos hacerlo con la siguiente abreviatura. `None` añade una nueva dimensión donde `:` selecciona todos los datos en un tensor.

In [ ]:
batch_test_matrix = test_matrix[None, :]
batch_test_matrix

In [ ]:
nn.Flatten()(batch_test_matrix)

El orden importa Esto es lo que ocurre cuando lo hacemos al revés:

In [ ]:
nn.Flatten()(test_matrix[:, None])

Ahora que ya le hemos cogido el truco a la capa «Aplanar», vamos a añadirla a nuestra lista de «capas».

In [ ]:
layers = [
    nn.Flatten()
]
layers

### 1.5.2 La capa de entrada

Nuestra primera capa de neuronas conecta nuestra imagen aplanada con el resto de nuestro modelo. Para ello, utilizaremos una capa [Lineal](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html). Esta capa estará *densamente conectada*, lo que significa que cada neurona en ella, y sus pesos, afectarán a cada neurona en la siguiente capa.

Para crear estos pesos, Pytorch necesita saber el tamaño de nuestras entradas y cuántas neuronas queremos crear.
Dado que hemos aplanado nuestras imágenes, el tamaño de nuestras entradas es el número de canales, el número de píxeles verticalmente y el número de píxeles horizontalmente multiplicados juntos.

In [ ]:
input_size = 1 * 28 * 28

Elegir el número correcto de neuronas es lo que pone la «ciencia» en la «ciencia de datos», ya que se trata de captar la complejidad estadística del conjunto de datos. Por ahora, utilizaremos 512 neuronas. Intente jugar con este valor más adelante para ver cómo afecta al entrenamiento y para empezar a desarrollar un sentido de lo que significa este número.

Aprenderemos más sobre las funciones de activación más adelante, pero por ahora, utilizaremos la función de activación [relu](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html), que en resumen, ayudará a nuestra red a aprender cómo hacer conjeturas más sofisticadas sobre los datos que si se le exigiera hacer conjeturas basadas en alguna función estrictamente lineal.

In [ ]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
]
layers

### 1.5.3 La capa oculta

Ahora añadiremos una capa lineal adicional densamente conectada. En la próxima lección veremos por qué añadir otro conjunto de neuronas puede ayudar a mejorar el aprendizaje. Al igual que la capa de entrada necesitaba conocer la forma de los datos que se le pasaban, la capa oculta [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html) necesita conocer la forma de los datos que se le pasan. Cada neurona de la capa anterior calculará un número, por lo que el número de entradas en la capa oculta es el mismo que el número de neuronas de la anterior.

In [ ]:
layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU()  # Activation for hidden
]
layers

### 1.5.4 La capa de salida

Por último, añadiremos una capa de salida. En este caso, como la red debe adivinar si una imagen pertenece a una de las 10 categorías posibles, habrá 10 salidas. A cada salida se le asigna una neurona. Cuanto mayor sea el valor de la neurona de salida en comparación con las demás neuronas, más predice el modelo que la imagen de entrada pertenece a la clase asignada a la neurona de salida.

No asignaremos la función `relu` a la capa de salida. En su lugar, aplicaremos una `función de pérdida` que veremos en la siguiente sección.

In [ ]:
n_classes = 10

layers = [
    nn.Flatten(),
    nn.Linear(input_size, 512),  # Input
    nn.ReLU(),  # Activation for input
    nn.Linear(512, 512),  # Hidden
    nn.ReLU(),  # Activation for hidden
    nn.Linear(512, n_classes)  # Output
]
layers

### 1.5.5 Compilación del modelo

Un modelo [Secuencial](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html) espera una secuencia de argumentos, no una lista, por lo que podemos utilizar el [operador *](https://docs.python.org/3/reference/expressions.html#expression-lists) para descomprimir nuestra lista de capas en una secuencia. Podemos imprimir el modelo para verificar que estas capas se han cargado correctamente.

In [ ]:
model = nn.Sequential(*layers)
model

Al igual que los tensores, cuando el modelo se inicializa por primera vez, se procesará en una CPU. Para que se procese con una GPU, podemos utilizar `to(device)`.

In [ ]:
model.to(device)

Para comprobar en qué dispositivo está un modelo, podemos comprobar en qué dispositivo están los parámetros del modelo. Echa un vistazo a este post [stack overflow](https://stackoverflow.com/questions/58926054/how-to-get-the-device-type-of-a-pytorch-module-conveniently) para obtener más información.

In [ ]:
next(model.parameters()).device

[PyTorch 2.0](https://pytorch.org/get-started/pytorch-2.0/) introdujo la capacidad de compilar un modelo para un rendimiento más rápido. Más información [aquí](https://pytorch.org/tutorials/intermediate/torch_compile_tutorial.html).

In [ ]:
model = torch.compile(model)

## 1.6 Entrenamiento del modelo

Ahora que hemos preparado datos de entrenamiento y validación, y un modelo, es hora de entrenar nuestro modelo con nuestros datos de entrenamiento, y verificarlo con sus datos de validación.

«Entrenar un modelo con datos» suele llamarse también “ajustar un modelo a los datos”. Dicho de otro modo, pone de relieve que la forma del modelo cambia con el tiempo para comprender con mayor precisión los datos que se le dan.

### 1.6.1 Pérdidas y optimización

Al igual que los profesores califican a sus alumnos, tenemos que proporcionar al modelo una función con la que calificar sus respuestas. Es lo que se denomina «función de pérdida». Usaremos un tipo de función de pérdida llamada [CrossEntropy](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) que está diseñada para calificar si un modelo predijo la categoría correcta de un grupo de categorías.

In [ ]:
loss_function = nn.CrossEntropyLoss()

A continuación, seleccionamos un «optimizador» para nuestro modelo. Si el `loss_function` proporciona un grado, el optimizador le dice al modelo cómo aprender de este grado para hacerlo mejor la próxima vez.

In [ ]:
optimizer = Adam(model.parameters())

### 1.6.2 Cálculo de la Exactitud


Aunque los resultados de la función de pérdida son eficaces para ayudar a nuestro modelo a aprender, los valores pueden ser difíciles de interpretar para los humanos. Por eso, los científicos de datos suelen incluir otras métricas como la exactitud.

Para calcular la exactitud con precisión, debemos comparar el número de clasificaciones correctas con el número total de predicciones realizadas. Dado que estamos mostrando datos al modelo en lotes, nuestra exactitud puede calcularse junto con estos lotes.

En primer lugar, el número total de predicciones tiene el mismo tamaño que nuestro conjunto de datos. Asignemos el tamaño de nuestros conjuntos de datos a `N`, donde `n` es sinónimo del `tamaño del lote`.

In [ ]:
train_N = len(train_loader.dataset)
valid_N = len(valid_loader.dataset)

A continuación, crearemos una función para calcular la exactitud de cada lote. El resultado es una fracción de la exactitud total, por lo que podemos sumar la exactitud de cada lote para obtener el total.

In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

### 1.6.3 La función Train

Aquí es donde todo se junta. A continuación se muestra la función que hemos definido para entrenar nuestro modelo basado en los datos de entrenamiento. Vamos a caminar a través de cada línea de código con más detalle más adelante, pero tome un momento para revisar cómo está estructurado. ¿Puedes reconocer las variables que hemos creado antes?

In [ ]:
def train():
    loss = 0
    accuracy = 0

    model.train()
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        output = model(x)
        optimizer.zero_grad()
        batch_loss = loss_function(output, y)
        batch_loss.backward()
        optimizer.step()

        loss += batch_loss.item()
        accuracy += get_batch_accuracy(output, y, train_N)
    print('Train - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

### 1.6.4 La función Validar

Del mismo modo, este es el código para validar el modelo con datos en los que no se ha entrenado. ¿Puedes detectar algunas diferencias con la función `train`?

In [ ]:
def validate():
    loss = 0
    accuracy = 0

    model.eval()
    with torch.no_grad():
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            output = model(x)

            loss += loss_function(output, y).item()
            accuracy += get_batch_accuracy(output, y, valid_N)
    print('Valid - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

### 1.6.5 El bucle de entrenamiento

Para ver cómo progresa el modelo, alternaremos entre entrenamiento y validación. De la misma forma que a un estudiante le puede llevar varias veces repasar las fichas para aprender todos los conceptos, el modelo repasará los datos de entrenamiento varias veces para comprenderlos cada vez mejor.

Una `epoca` es una pasada completa por todo el conjunto de datos. Vamos a entrenar y validar el modelo durante 5 `epocas` para ver cómo aprende.

In [ ]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

epochs = 5

for epoch in range(epochs):
    print('Epoch: {}'.format(epoch))
    train()
    validate()

¡Ya estamos cerca del 100%! Veamos si es cierto probándolo en nuestra muestra original. Podemos utilizar nuestro modelo como una función:

In [ ]:
prediction = model(x_0_gpu)
prediction

Debería haber diez números, cada uno correspondiente a una neurona de salida distinta. Gracias a la estructura de los datos, el índice de cada número coincide con el número manuscrito correspondiente. El índice 0 es una predicción para un 0 escrito a mano, el índice 1 es una predicción para un 1 escrito a mano, y así sucesivamente.

Podemos utilizar la función `argmax` para encontrar el índice del valor más alto.

In [ ]:
prediction.argmax(dim=1, keepdim=True)

¿Lo ha hecho bien?

In [ ]:
y_0

## 1.7 Resumen

El modelo funcionó bastante bien. La exactitud alcanzó rápidamente cerca del 100%, al igual que la exactitud de validación. Ahora tenemos un modelo que puede utilizarse para detectar y clasificar con exactitud imágenes manuscritas.

El siguiente paso sería utilizar este modelo para clasificar nuevas imágenes manuscritas aún no vistas. Esto se denomina [inferencia](https://blogs.nvidia.com/blog/2016/08/22/difference-deep-learning-training-inference-ai/). Exploraremos el proceso de inferencia en un ejercicio posterior.

Merece la pena tomarse un momento para apreciar lo que hemos hecho aquí. Históricamente, los sistemas expertos que se construían para realizar este tipo de tareas eran extremadamente complicados, y la gente se pasaba la carrera construyéndolos (echa un vistazo a las referencias en la [página oficial de MNIST](http://yann.lecun.com/exdb/mnist/) y a los años en que se alcanzaron hitos).

MNIST no sólo es útil por su influencia histórica en la visión por ordenador, sino que también es un gran [punto de referencia](http://www.cs.toronto.edu/~serailhydra/publications/tbd-iiswc18.pdf) y una herramienta de depuración. ¿Tienes problemas para hacer funcionar una nueva arquitectura de aprendizaje automático? Compárela con MNIST. Si no puede aprender en este conjunto de datos, lo más probable es que no aprenda en imágenes y conjuntos de datos más complicados.

### 1.7.2 Siguiente

En esta sección has aprendido a construir y entrenar una red neuronal sencilla para la clasificación de imágenes. En la siguiente sección, se le pedirá que construya su propia red neuronal y realice la preparación de datos para resolver un problema diferente de clasificación de imágenes.

En base alo epxlicado en el notebook, ¿Cómo el Deep Learning permite resolver problemas que las técnicas tradicionales no pueden?
